In [1]:
import requests
import re
import json
import os
import pandas as pd
import sys
import numpy as np

# project lib
PROJECT_SRC_PATH = os.path.join( '/workspace/workspace/ufo-prediction', 'src-RCA-UFO')
sys.path.append(PROJECT_SRC_PATH)
import dataset

In [2]:
# Set a seed for reproducibility
np.random.seed(42)  # You can choose any number as your seed

# Define paths for data
path_data_NLD = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-NLD.pkl')
path_data_FRA = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-FRA.pkl')
path_data_ESP = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-ESP.pkl')
processed_df_NLD = "/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD_adv.csv"
processed_df_ESP = "/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv"
processed_df_FRA = "/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA_adv.csv"
processed_df_ALL = "/workspace/workspace/ufo-prediction/demo/kartaview_key.csv"

# Image directory paths
image_dir_map = {
    'NLD': '/workspace/workspace/ufo-prediction/image_data_NLD_adv',
    'FRA': '/workspace/workspace/ufo-prediction/image_data_FRA_adv',
    'ESP': '/workspace/workspace/ufo-prediction/image_data_ESP_adv',
    'ALL': '/workspace/workspace/ufo-prediction/image_data'
}

# Ask for user input
country_code = input("Enter country code (FRA, NLD, ESP) or ALL: ").upper()

# Map user input to the correct path
path_data_map = {
    'NLD': processed_df_NLD,
    'FRA': processed_df_FRA,
    'ESP': processed_df_ESP,
    'ALL': processed_df_ALL
}

raw_data_path_map = {
'NLD': path_data_NLD,
'FRA': path_data_FRA,
'ESP': path_data_ESP
}
# Check if the input is valid
if country_code not in path_data_map:
    print("Invalid country code or specification. Please enter FRA, NLD, ESP, or ALL.")
else:
    processed_df_path = path_data_map[country_code]
    # Set directory based on country code
    current_directory = image_dir_map[country_code]

    # Check if the new directory exists, if not, create it
    if not os.path.exists(current_directory):
        os.makedirs(current_directory)
        print(f"Directory {current_directory} created.")
    else:
        print(f"Directory {current_directory} already exists.")

    # Process for ALL
    if country_code == 'ALL':
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print("Creating a new processed DataFrame for ALL")
            path_data_RCA = os.path.join(dataset.DATA_DIR, 'rca-ufo-merge_ALL.csv')
            df = pd.read_csv(path_data_RCA, encoding='latin1')
            kartaview_keys = df[['lon', 'lat','age_right', 'id', 'PropertyKey_ID']]
            kartaview_keys.to_csv(processed_df_path, index=False)
            

    # Process for FRA, NLD, ESP
    else:
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print(f"Creating a new processed DataFrame for {country_code}")
            df_path = raw_data_path_map[country_code]
            df = pd.read_pickle(df_path)
            print("Loaded DataFrame from", df_path)
            sampled_df = df.sample(n=1500000, random_state=42)
            kartaview_keys = sampled_df[['lon', 'lat', 'age', 'id']].rename(columns={'age': 'age_right'})
            kartaview_keys.to_csv(processed_df_path, index=False)

print(kartaview_keys) 

Enter country code (FRA, NLD, ESP) or ALL: ESP
Directory /workspace/workspace/ufo-prediction/image_data_ESP_adv already exists.
Loaded processed DataFrame from /workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv
             lon        lat  age_right                         id
0      -5.880051  37.594850     1975.0   v0.1-ESP.1.8.3.18_1-1077
1      -0.813850  38.245625     1980.0   v0.1-ESP.10.1.9.2_1-8403
2      -6.352885  36.766776     2006.0   v0.1-ESP.1.2.4.3_1-12777
3      -5.546561  37.261420     1972.0    v0.1-ESP.1.8.2.2_1-7638
4      -1.587417  37.407919     1989.0   v0.1-ESP.18.1.2.1_1-5190
...          ...        ...        ...                        ...
918909 -5.984130  37.675687     2011.0    v0.1-ESP.1.8.1.4_1-5558
918910  2.505601  41.879092     1900.0    v0.1-ESP.6.2.6.19_1-823
918911 -3.924419  37.463344     1980.0   v0.1-ESP.1.6.9.1_1-10785
918912 -0.220692  38.513329     1945.0  v0.1-ESP.10.1.7.17_1-5766
918913 -4.000000        NaN        NaN         

In [3]:
print(current_directory)
print(kartaview_keys)

/workspace/workspace/ufo-prediction/image_data_ESP_adv
             lon        lat  age_right                         id
0      -5.880051  37.594850     1975.0   v0.1-ESP.1.8.3.18_1-1077
1      -0.813850  38.245625     1980.0   v0.1-ESP.10.1.9.2_1-8403
2      -6.352885  36.766776     2006.0   v0.1-ESP.1.2.4.3_1-12777
3      -5.546561  37.261420     1972.0    v0.1-ESP.1.8.2.2_1-7638
4      -1.587417  37.407919     1989.0   v0.1-ESP.18.1.2.1_1-5190
...          ...        ...        ...                        ...
918909 -5.984130  37.675687     2011.0    v0.1-ESP.1.8.1.4_1-5558
918910  2.505601  41.879092     1900.0    v0.1-ESP.6.2.6.19_1-823
918911 -3.924419  37.463344     1980.0   v0.1-ESP.1.6.9.1_1-10785
918912 -0.220692  38.513329     1945.0  v0.1-ESP.10.1.7.17_1-5766
918913 -4.000000        NaN        NaN                        NaN

[918914 rows x 4 columns]


In [ ]:
# Assuming current_directory and processed_df_path are set from the previous code chunk
print(f"Images will be saved in: {current_directory}")
print(f"Using kartaview_keys from: {processed_df_path}")

# Consistency check between image directory and kartaview_keys path
expected_csv_map = {
    '/workspace/workspace/ufo-prediction/image_data': '/workspace/workspace/ufo-prediction/demo/kartaview_key.csv',
    '/workspace/workspace/ufo-prediction/image_data_NLD': '/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD.csv',
    '/workspace/workspace/ufo-prediction/image_data_FRA': '/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv',
    '/workspace/workspace/ufo-prediction/image_data_ESP': '/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP.csv',
}

# Stop the code if using image_data directory
if current_directory == '/workspace/workspace/ufo-prediction/image_data':
    print("Download for the 'image_data' directory has already been completed. Stopping execution.")
    # Use `exit()` or `sys.exit()` depending on your environment
    exit()

if processed_df_path != expected_csv_map.get(current_directory):
    print("Inconsistency detected between the image directory and the kartaview_keys path. Please check.")
    exit()

image_count = {} 
print("Number of buildings remaining: ",len(kartaview_keys))

#Initialise a counter for the loop iterations
iteration_counter = 0

for index, row in kartaview_keys.iterrows():
    iteration_counter += 1 # Increment the counter with each iteration
    precision = 6  # Start with 6 decimal places
    success = False  # Flag to indicate if the request was successful

    while precision > 2 and not success:
        # Format lon and lat to the current precision
        lon = f"{row['lon']:.{precision}f}"
        lat = f"{row['lat']:.{precision}f}"

        # Construct the API URL
        url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}".format(lat, lon)

        # Send a GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'
            urls = re.findall(url_pattern, json.dumps(data))

            filtered_urls = [
                url for url in urls
                if not any(x in url.rsplit('/', 2)[-2] for x in ["{{sizeprefix}}", "proc"]) and
                ("th" in url.rsplit('/', 2)[-2] and not "lth" in url.rsplit('/', 2)[-2])
            ]

            if filtered_urls:
                # Initialize or update the image count for the current ID
                building_id = row['id']  # Assuming 'id' column exists in your DataFrame
                if building_id not in image_count:
                    image_count[building_id] = 0

                for image_url in filtered_urls:
                    image_count[building_id] += 1  # Increment the image count for the building
                    subscript = image_count[building_id]  # Subscript for the file name
                    file_name = f"{row['age_right']}_{building_id}_{subscript}.jpg"
                    file_path = os.path.join(current_directory, file_name)

                    # Check if the file already exists
                    if os.path.exists(file_path):
                        print(f"File already exists: {file_path}. Skipping download.")
                    else:
                        image_response = requests.get(image_url)

                        if image_response.status_code == 200:
                            with open(file_path, 'wb') as f:
                                f.write(image_response.content)
                            print("Image downloaded successfully: {}".format(file_path))
                        else:
                            print("Failed to download the image.")
                success = True  # Mark success as True to exit the while loop
            else:
                print("No suitable images found for location: lon={}, lat={}".format(lon, lat))
                precision -= 1  # Reduce precision by one decimal place
        else:
            print("Failed to retrieve data from the API for location: lon={}, lat={}. Trying with reduced precision.")
            

    # After processing, remove the row from df_subset
    kartaview_keys = kartaview_keys.drop(index)

    # Only save the updated DataFrame to a CSV file every 100th instance
    if iteration_counter % 100 == 0:
        print(f"Saving progress at iteration {iteration_counter} to {processed_df_path}. ")
        print("Number of buildings remaining: ",len(kartaview_keys))
        kartaview_keys.to_csv(processed_df_path, index=False)

    if not success:
        print("Unable to retrieve data from the API with sufficient precision for location: lon={}, lat={}".format(row['lon'], row['lat']))
        
if iteration_counter % 100 != 0:
    print(f"Saving final progress")
    print("Number of buildings remaining: ",len(kartaview_keys))
    kartaview_keys.to_csv(processed_df_path, index=False)

Images will be saved in: /workspace/workspace/ufo-prediction/image_data_ESP_adv
Using kartaview_keys from: /workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv
Inconsistency detected between the image directory and the kartaview_keys path. Please check.
Number of buildings remaining:  918914
No suitable images found for location: lon=-5.880051, lat=37.594850
No suitable images found for location: lon=-5.88005, lat=37.59485
No suitable images found for location: lon=-5.8801, lat=37.5949
No suitable images found for location: lon=-5.880, lat=37.595
Unable to retrieve data from the API with sufficient precision for location: lon=-5.880050793276933, lat=37.59485021591763
No suitable images found for location: lon=-0.813850, lat=38.245625
No suitable images found for location: lon=-0.81385, lat=38.24563
No suitable images found for location: lon=-0.8139, lat=38.2456
No suitable images found for location: lon=-0.814, lat=38.246
Unable to retrieve data from the API with sufficie

No suitable images found for location: lon=-1.512331, lat=39.090962
No suitable images found for location: lon=-1.51233, lat=39.09096
No suitable images found for location: lon=-1.5123, lat=39.0910
No suitable images found for location: lon=-1.512, lat=39.091
Unable to retrieve data from the API with sufficient precision for location: lon=-1.512331226591845, lat=39.09096188282897
No suitable images found for location: lon=-8.674620, lat=42.236213
No suitable images found for location: lon=-8.67462, lat=42.23621
No suitable images found for location: lon=-8.6746, lat=42.2362
No suitable images found for location: lon=-8.675, lat=42.236
Unable to retrieve data from the API with sufficient precision for location: lon=-8.674619816329434, lat=42.23621274871382
No suitable images found for location: lon=2.125748, lat=41.361081
No suitable images found for location: lon=2.12575, lat=41.36108
No suitable images found for location: lon=2.1257, lat=41.3611
No suitable images found for location: 

No suitable images found for location: lon=-0.2722, lat=38.6087
No suitable images found for location: lon=-0.272, lat=38.609
Unable to retrieve data from the API with sufficient precision for location: lon=-0.2721710703758434, lat=38.608718511703536
No suitable images found for location: lon=2.691070, lat=41.627603
No suitable images found for location: lon=2.69107, lat=41.62760
No suitable images found for location: lon=2.6911, lat=41.6276
No suitable images found for location: lon=2.691, lat=41.628
Unable to retrieve data from the API with sufficient precision for location: lon=2.691069896682577, lat=41.62760306454695
No suitable images found for location: lon=-1.663125, lat=38.080157
No suitable images found for location: lon=-1.66312, lat=38.08016
No suitable images found for location: lon=-1.6631, lat=38.0802
No suitable images found for location: lon=-1.663, lat=38.080
Unable to retrieve data from the API with sufficient precision for location: lon=-1.663124930720294, lat=38.080

No suitable images found for location: lon=-4.480078, lat=36.755080
No suitable images found for location: lon=-4.48008, lat=36.75508
No suitable images found for location: lon=-4.4801, lat=36.7551
No suitable images found for location: lon=-4.480, lat=36.755
Unable to retrieve data from the API with sufficient precision for location: lon=-4.480077540364613, lat=36.75508028772303
No suitable images found for location: lon=-3.970181, lat=39.886652
No suitable images found for location: lon=-3.97018, lat=39.88665
No suitable images found for location: lon=-3.9702, lat=39.8867
No suitable images found for location: lon=-3.970, lat=39.887
Unable to retrieve data from the API with sufficient precision for location: lon=-3.970180506905312, lat=39.886651836232645
No suitable images found for location: lon=0.119460, lat=42.036476
No suitable images found for location: lon=0.11946, lat=42.03648
No suitable images found for location: lon=0.1195, lat=42.0365
No suitable images found for location:

No suitable images found for location: lon=-8.626396, lat=42.625968
No suitable images found for location: lon=-8.62640, lat=42.62597
No suitable images found for location: lon=-8.6264, lat=42.6260
No suitable images found for location: lon=-8.626, lat=42.626
Unable to retrieve data from the API with sufficient precision for location: lon=-8.62639581676528, lat=42.62596815931279
No suitable images found for location: lon=-3.873455, lat=43.055157
No suitable images found for location: lon=-3.87345, lat=43.05516
No suitable images found for location: lon=-3.8735, lat=43.0552
No suitable images found for location: lon=-3.873, lat=43.055
Unable to retrieve data from the API with sufficient precision for location: lon=-3.873454707905304, lat=43.055156758347934
No suitable images found for location: lon=-14.018846, lat=28.421784
No suitable images found for location: lon=-14.01885, lat=28.42178
No suitable images found for location: lon=-14.0188, lat=28.4218
No suitable images found for loca

...

No suitable images found for location: lon=-1.799, lat=39.239
Unable to retrieve data from the API with sufficient precision for location: lon=-1.79909210189276, lat=39.238912545582735
No suitable images found for location: lon=-4.192573, lat=40.242427
No suitable images found for location: lon=-4.19257, lat=40.24243
No suitable images found for location: lon=-4.1926, lat=40.2424
No suitable images found for location: lon=-4.193, lat=40.242
Unable to retrieve data from the API with sufficient precision for location: lon=-4.192572608519161, lat=40.24242686129369
No suitable images found for location: lon=-5.819163, lat=43.633435
No suitable images found for location: lon=-5.81916, lat=43.63343
No suitable images found for location: lon=-5.8192, lat=43.6334
No suitable images found for location: lon=-5.819, lat=43.633
Unable to retrieve data from the API with sufficient precision for location: lon=-5.819163436816889, lat=43.63343492691604
No suitable images found for location: lon=-2.652

No suitable images found for location: lon=3.058618, lat=39.726197
No suitable images found for location: lon=3.05862, lat=39.72620
No suitable images found for location: lon=3.0586, lat=39.7262
No suitable images found for location: lon=3.059, lat=39.726
Unable to retrieve data from the API with sufficient precision for location: lon=3.058618022510259, lat=39.72619720939457
No suitable images found for location: lon=-3.361024, lat=38.014820
No suitable images found for location: lon=-3.36102, lat=38.01482
No suitable images found for location: lon=-3.3610, lat=38.0148
No suitable images found for location: lon=-3.361, lat=38.015
Unable to retrieve data from the API with sufficient precision for location: lon=-3.3610238348194312, lat=38.01481966789007
No suitable images found for location: lon=-5.139207, lat=36.797252
No suitable images found for location: lon=-5.13921, lat=36.79725
No suitable images found for location: lon=-5.1392, lat=36.7973
No suitable images found for location: l

No suitable images found for location: lon=-1.17012, lat=37.56214
No suitable images found for location: lon=-1.1701, lat=37.5621
No suitable images found for location: lon=-1.170, lat=37.562
Unable to retrieve data from the API with sufficient precision for location: lon=-1.1701162176199391, lat=37.56213960464955
No suitable images found for location: lon=-4.069826, lat=40.245301
No suitable images found for location: lon=-4.06983, lat=40.24530
No suitable images found for location: lon=-4.0698, lat=40.2453
No suitable images found for location: lon=-4.070, lat=40.245
Unable to retrieve data from the API with sufficient precision for location: lon=-4.069825507172067, lat=40.24530116196743
No suitable images found for location: lon=-5.746299, lat=36.813120
No suitable images found for location: lon=-5.74630, lat=36.81312
No suitable images found for location: lon=-5.7463, lat=36.8131
No suitable images found for location: lon=-5.746, lat=36.813
Unable to retrieve data from the API with

No suitable images found for location: lon=-8.131350, lat=42.683738
No suitable images found for location: lon=-8.13135, lat=42.68374
No suitable images found for location: lon=-8.1314, lat=42.6837
No suitable images found for location: lon=-8.131, lat=42.684
Unable to retrieve data from the API with sufficient precision for location: lon=-8.131350150067506, lat=42.68373760374996
No suitable images found for location: lon=-8.545303, lat=42.295102
No suitable images found for location: lon=-8.54530, lat=42.29510
No suitable images found for location: lon=-8.5453, lat=42.2951
No suitable images found for location: lon=-8.545, lat=42.295
Unable to retrieve data from the API with sufficient precision for location: lon=-8.545303225423334, lat=42.2951021754437
No suitable images found for location: lon=1.621106, lat=41.515704
No suitable images found for location: lon=1.62111, lat=41.51570
No suitable images found for location: lon=1.6211, lat=41.5157
No suitable images found for location: l

No suitable images found for location: lon=-2.693, lat=40.140
Unable to retrieve data from the API with sufficient precision for location: lon=-2.692910749631378, lat=40.14014683104867
No suitable images found for location: lon=-8.176105, lat=42.890303
No suitable images found for location: lon=-8.17610, lat=42.89030
No suitable images found for location: lon=-8.1761, lat=42.8903
No suitable images found for location: lon=-8.176, lat=42.890
Unable to retrieve data from the API with sufficient precision for location: lon=-8.176104501726872, lat=42.89030261845776
No suitable images found for location: lon=-7.018845, lat=43.537159
No suitable images found for location: lon=-7.01885, lat=43.53716
No suitable images found for location: lon=-7.0188, lat=43.5372
No suitable images found for location: lon=-7.019, lat=43.537
Unable to retrieve data from the API with sufficient precision for location: lon=-7.018845086379562, lat=43.53715879842721
No suitable images found for location: lon=-7.979